# 2022-10-27

In [1]:
import requests
from bs4 import BeautifulSoup

from typing import List, Dict, Optional

import pandas as pd

In [2]:
page = requests.get("https://www.lamaistas.lt/visi-receptai")

In [3]:
page.status_code

200

In [4]:
soup = BeautifulSoup(page.content, 'html.parser')

In [11]:
type(soup)

bs4.BeautifulSoup

In [ ]:
print(soup.prettify())

In [6]:
all_ps = soup.find_all("p")

In [7]:
all_ps

[<p>Turinio klausimais rašykite:</p>,
 <p><a href="/cdn-cgi/l/email-protection#691a0208071c2905080408001a1d081a47051d"><span class="__cf_email__" data-cfemail="fa89919b948fba969b979b93898e9b89d4968e">[email protected]</span></a></p>,
 <p>(darbo laikas nuo 8 iki 17 val.)</p>,
 <p>Reklamos klausimais rašykite</p>,
 <p><a href="/cdn-cgi/l/email-protection#1b697e70777a767a5b7a767f727c726f7a7735776f"><span class="__cf_email__" data-cfemail="285a4d43444945496849454c414f415c494406445c">[email protected]</span></a></p>,
 <p>(darbo laikas nuo 8 iki 17 val.)</p>]

In [9]:
all_divs = soup.find_all("div")

In [10]:
print(type(all_ps[0]))
print(all_ps[0].text)

<class 'bs4.element.Tag'>
Turinio klausimais rašykite:


**labas**
# h1
## h2

<h1></h1>
<h2></h2>
<b>labas</b>

In [48]:
recipes_list_div = soup.find("div", class_="recipesResultSegment")
all_recipes_div = recipes_list_div.find_all("div", class_="frame")

results: List[Dict] = []
for recipe_div in all_recipes_div:
    title = recipe_div.find("div", class_="title").text
    link_to_recipe = recipe_div.find("a")["href"]
    link_to_image = recipe_div.find("img")["src"]

    results.append({
        "title": title.strip(), 
        "link_to_recipe": link_to_recipe,
        "link_to_image": link_to_image
    })

results_df = pd.DataFrame(results)
results_df.to_csv("lamaistas.csv")

In [58]:
def extract_recipes_info(url: str) -> Optional[List[Dict]]:
    resp = requests.get(url)
    if resp.status_code != 200:
        raise Exception("Cannot reach page!")
    soup = BeautifulSoup(resp.content, 'html.parser')
    recipes_list_div = soup.find("div", class_="recipesResultSegment")

    if not recipes_list_div:
        return None    
    all_recipes_div = recipes_list_div.find_all("div", class_="frame")
    results: List[Dict] = []
    for recipe_div in all_recipes_div:
        title = recipe_div.find("div", class_="title").text
        link_to_recipe = recipe_div.find("a")["href"]
        link_to_image = recipe_div.find("img")["src"]

        results.append({
            "title": title.strip(), 
            "link_to_recipe": link_to_recipe,
            "link_to_image": link_to_image
        })

    return results

In [92]:
recipes_links: List[Dict] = []

for page in range(2, 5):
    url = f"https://www.lamaistas.lt/visi-receptai/{page}"
    recipes_links.extend(extract_recipes_info(url))

pd.DataFrame(recipes_links).to_csv("lamaistas.csv")

In [13]:
soup.find("div")['src']

KeyError: 'src'

## Info is puslapio

In [66]:
resp = requests.get("https://www.lamaistas.lt/receptas/naminiu-ledu-sumustiniai-su-braskemis-arba-kitomis-uogomis-81350")
soup = BeautifulSoup(resp.content, "html.parser")

recipe_title = soup.find("div", class_="recipeTitleSegment").find("h1").text
main_recipe_image = soup.find("div", class_="bigImg").find("img").get("src")
about_recipe = soup.find("div", class_="authorAboutRecipeSegment").text

'https://www.lamaistas.lt/uploads/modules/recipes/without-watermark/40168.jpg'

In [79]:
ingredients_div = soup.find("div", class_="ingredients")
ingredients_table = ingredients_div.find("table")
tr_rows = ingredients_table.find_all("tr")
ingredients: List[Dict] = []
for tr_row in tr_rows:
    spans = tr_row.find_all("span")
    ingredients.append({"ingredient": spans[1].text.strip(), "ammount": spans[0].text.strip()})

In [83]:
making_time = soup.find("div", class_="method").find("div", class_="info").text.strip().replace("Paruošimo laikas: ", "")

In [86]:
making_steps: List[Dict] = []
description_divs = soup.find("div", class_="method").find("div", class_="infoA").find_all("div", class_="description")
for step, description_div in enumerate(description_divs):
    making_steps.append({"step": step + 1, "description": description_div.find("div", class_="text").text})


In [94]:
def get_recipe_info(url: str) -> Dict:
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")

    recipe_title = soup.find("div", class_="recipeTitleSegment").find("h1").text
    main_recipe_image = soup.find("div", class_="bigImg").find("img").get("src")
    about_recipe = soup.find("div", class_="authorAboutRecipeSegment").text

    ingredients_div = soup.find("div", class_="ingredients")
    ingredients_table = ingredients_div.find("table")
    tr_rows = ingredients_table.find_all("tr")
    ingredients: List[Dict] = []
    try:
        for tr_row in tr_rows:
            spans = tr_row.find_all("span")
            ingredients.append({"ingredient": spans[1].text.strip(), "ammount": spans[0].text.strip()})
    except IndexError:
        pass

    making_time = soup.find("div", class_="method").find("div", class_="info").text.strip().replace("Paruošimo laikas: ", "")
    making_steps: List[Dict] = []
    description_divs = soup.find("div", class_="method").find("div", class_="infoA").find_all("div", class_="description")
    for step, description_div in enumerate(description_divs):
        making_steps.append({"step": step + 1, "description": description_div.find("div", class_="text").text})

    return {
        "title": recipe_title.strip(),
        "image": main_recipe_image,
        "about": about_recipe.strip(),
        "making_time": making_time,
        "ingredients": ingredients,
        "making_steps": making_steps
    }

In [96]:
full_recipes: List[Dict] = []

for recipe_link in recipes_links:
    try:
        full_recipes.append(get_recipe_info(recipe_link["link_to_recipe"]))
    except:
        print(recipe_link)

{'title': 'Obuolių pyragas „Šarlotė“', 'link_to_recipe': 'https://www.lamaistas.lt/receptas/obuoliu-pyragas-sarlote-81758', 'link_to_image': 'https://www.lamaistas.lt/uploads/modules/recipes/thumb300x224/41383.jpg'}
{'title': 'Burokėlių ir pomidorų mišrainė žiemai', 'link_to_recipe': 'https://www.lamaistas.lt/receptas/burokeliu-ir-pomidoru-misraine-ziemai-10337', 'link_to_image': 'https://www.lamaistas.lt/uploads/modules/recipes/thumb300x224/14604.jpg'}
{'title': 'Bulviniai lietiniai itališkai', 'link_to_recipe': 'https://www.lamaistas.lt/receptas/bulviniai-lietiniai-italiskai-81745', 'link_to_image': 'https://www.lamaistas.lt/uploads/modules/recipes/thumb300x224/41203.jpg'}
{'title': 'Drėgnas šokoladinis moliūgų pyragas', 'link_to_recipe': 'https://www.lamaistas.lt/receptas/dregnas-sokoladinis-moliugu-pyragas-58459', 'link_to_image': 'https://www.lamaistas.lt/uploads/modules/recipes/thumb300x224/17722.jpg'}
{'title': 'Burokėlių sriuba su mėsos kukuliukais', 'link_to_recipe': 'https://

In [98]:
len(full_recipes)

107